<a href="https://colab.research.google.com/github/scarlettyu2023/AI_agent_workshop/blob/main/Topic5RAG/manual_rag_pipeline_universal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manual RAG Pipeline: Mechanisms First

This notebook builds a Retrieval-Augmented Generation (RAG) pipeline from scratch.
You'll see every step explicitly before we move to frameworks like LangChain.

**Works on:** Google Colab, Local Jupyter (Mac/Windows/Linux)

**Pipeline Overview:**
```
Documents â†’ Chunking â†’ Embedding â†’ Index (FAISS)
                                        â†“
User Query â†’ Embed Query â†’ Similarity Search â†’ Top-K Chunks
                                                    â†“
                                        Prompt Assembly â†’ LLM â†’ Answer
```

## TODO â€” Topic 5 RAG Course Project Checklist

- **Exercise 0:** Set-up â€” Get notebook running; unzip Corpora.zip. Use PDFs from `Corpora/<corpus>/pdf_embedded/`.
- **Exercise 1:** Open model RAG vs no RAG â€” Compare Qwen 2.5 1.5B with/without RAG on Model T manual and Congressional Record.
- **Exercise 2:** Open model + RAG vs large model â€” Run GPT-4o Mini with no tools on same queries.
- **Exercise 3:** Open model + RAG vs frontier chat â€” Compare local Qwen+RAG vs GPT-4/Claude (web).
- **Exercise 4:** Effect of top-K â€” Test k = 1, 3, 5, 10, 20.
- **Exercise 5:** Unanswerable questions â€” Off-topic, related-but-missing, false premise.
- **Exercise 6:** Query phrasing sensitivity â€” Same question in 5+ phrasings.
- **Exercise 7:** Chunk overlap â€” Re-chunk with overlap 0, 64, 128, 256.
- **Exercise 8:** Chunk size â€” Chunk at 128, 256, 512, 1024, 2048.
- **Exercise 9:** Retrieval score analysis â€” 10 queries, top-10 chunks, score distribution.
- **Exercise 10:** Prompt template variations â€” Minimal, strict grounding, citation, permissive, structured.
- **Exercise 11:** Failure mode catalog â€” Computation, temporal, comparison, ambiguous, multi-hop, etc.
- **Exercise 12:** Cross-document synthesis â€” Questions needing multiple chunks.

## Setup

First, let's install the required packages and detect our compute environment.

In [1]:
# Install dependencies
# On Colab, these install quickly. Locally, you may already have them.
# Use a kernel-aware install when available; fall back to subprocess otherwise.
try:
    ip = get_ipython()
    ip.run_line_magic('pip', 'install -q torch transformers sentence-transformers faiss-cpu pymupdf accelerate ipyfilechooser')
except NameError:
    import subprocess, sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'torch', 'transformers', 'sentence-transformers', 'faiss-cpu', 'pymupdf', 'accelerate', 'ipyfilechooser'])
# For Exercise 2 (GPT-4o Mini): add 'openai' to the list above if needed


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.4 MB/s eta 0:00:00


In [2]:
# =============================================================================
# ENVIRONMENT AND DEVICE DETECTION
# =============================================================================
import os
import sys

# Enable MPS fallback for any PyTorch operations not yet implemented on Metal
# This MUST be set before importing torch
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

# Prevent kernel crash from duplicate OpenMP libraries (PyTorch + FAISS conflict on macOS)
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import torch
from typing import Tuple

def detect_environment() -> str:
    """Detect if we're running on Colab or locally."""
    try:
        import google.colab
        return 'colab'
    except ImportError:
        return 'local'

def get_device() -> Tuple[str, torch.dtype]:
    """
    Detect the best available compute device.

    Priority: CUDA > MPS (Apple Silicon) > CPU

    Returns:
        Tuple of (device_string, recommended_dtype)

    Notes:
        - CUDA: Use float16 for memory efficiency (Tensor Cores optimize this)
        - MPS: Use float32 - Apple Silicon doesn't have the same float16
               optimizations as NVIDIA, and float32 is often faster
        - CPU: Use float32 (float16 not well supported on CPU)
    """
    if torch.cuda.is_available():
        device = 'cuda'
        dtype = torch.float16
        device_name = torch.cuda.get_device_name(0)
        memory_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"âœ“ Using CUDA GPU: {device_name} ({memory_gb:.1f} GB)")

    elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
        device = 'mps'
        dtype = torch.float32  # float32 is often faster on Apple Silicon!
        print("âœ“ Using Apple Silicon GPU (MPS)")
        print("  Note: Using float32 (faster than float16 on Apple Silicon)")

    else:
        device = 'cpu'
        dtype = torch.float32
        print("âš  Using CPU (no GPU detected)")
        print("  Tip: For faster processing, use a machine with a GPU")

    return device, dtype

# Detect environment and device
ENVIRONMENT = detect_environment()
DEVICE, DTYPE = get_device()

print(f"\nEnvironment: {ENVIRONMENT.upper()}")
print(f"Device: {DEVICE}, Dtype: {DTYPE}")

âš  Using CPU (no GPU detected)
  Tip: For faster processing, use a machine with a GPU

Environment: COLAB
Device: cpu, Dtype: torch.float32


## Load Your Documents

**Cell 1:** Configure your document source and select/upload files
- **Local Jupyter**: Use the folder picker, then run Cell 2
- **Colab + Upload**: Files upload immediately (blocking), then run Cell 2
- **Colab + Drive**: Set `USE_GOOGLE_DRIVE = True`, mounts Drive and shows picker, then run Cell 2

**Cell 2:** Confirms selection and lists documents

In [3]:
# =============================================================================
# CELL 1: SELECT DOCUMENT SOURCE
# =============================================================================
# This cell either:
#   - Shows a folder picker (Local or Colab+Drive) - NON-BLOCKING
#   - Shows an upload dialog (Colab+Upload) - BLOCKING
#
# If a folder picker is shown, SELECT YOUR FOLDER BEFORE running Cell 2.
# The picker widget is non-blocking, so the code continues before you select.
# =============================================================================

from pathlib import Path

# ------------- COLAB USERS: CONFIGURE HERE -------------
USE_GOOGLE_DRIVE = True  # Set to True to use Google Drive instead of uploading
# -------------------------------------------------------

# Default folder: use Corpora from course project (unzip Corpora.zip first).
_folder_default = Path("Corpora/ModelTService")
DOC_FOLDER = str(_folder_default) if _folder_default.exists() else "documents"
folder_chooser = None  # Will hold the picker widget if used

if ENVIRONMENT == 'colab':
    if USE_GOOGLE_DRIVE:
        # ----- COLAB + GOOGLE DRIVE -----
        # Mount Drive first, then show folder picker
        from google.colab import drive
        print("Mounting Google Drive...")
        drive.mount('/content/drive')
        print("âœ“ Google Drive mounted\n")

        # Now show folder picker for the Drive
        try:
            from ipyfilechooser import FileChooser

            folder_chooser = FileChooser(
                path='/content/drive/MyDrive',
                title='Select your documents folder in Google Drive',
                show_only_dirs=True,
                select_default=True
            )
            print("ðŸ“ Select your documents folder below, then run Cell 2:")
            print("   (The picker is non-blocking - select BEFORE running the next cell)")
            display(folder_chooser)

        except ImportError:
            # Fallback: manual path entry
            print("Folder picker not available.")
            print("Edit DOC_FOLDER below with your Google Drive path, then run Cell 2:")
            DOC_FOLDER = '/content/drive/MyDrive/your_documents_folder'  # â† Edit this!
            print(f"  DOC_FOLDER = '{DOC_FOLDER}'")
    else:
        # ----- COLAB + UPLOAD -----
        # Upload dialog blocks until complete, so DOC_FOLDER is ready when done
        from google.colab import files
        os.makedirs(DOC_FOLDER, exist_ok=True)

        print("Upload your documents (PDF, TXT, or MD):")
        print("(This dialog blocks until upload is complete)\n")
        uploaded = files.upload()

        for filename in uploaded.keys():
            os.rename(filename, f'{DOC_FOLDER}/{filename}')
            print(f"  âœ“ Saved: {DOC_FOLDER}/{filename}")

        print(f"\nâœ“ Upload complete. Run Cell 2 to continue.")

else:
    # ----- LOCAL JUPYTER -----
    # Show folder picker
    print("Running locally\n")

    try:
        from ipyfilechooser import FileChooser

        folder_chooser = FileChooser(
            path=str(Path.home()),
            title='Select your documents folder',
            show_only_dirs=True,
            select_default=True
        )
        print("ðŸ“ Select your documents folder below, then run Cell 2:")
        print("   (The picker is non-blocking - select BEFORE running the next cell)")
        display(folder_chooser)

    except ImportError:
        # Fallback: manual path entry
        print("Folder picker not available (ipyfilechooser not installed).")
        print(f"\nUsing default folder: {Path(DOC_FOLDER).absolute()}")
        print("\nTo use a different folder, edit DOC_FOLDER in this cell:")
        print("  DOC_FOLDER = '/path/to/your/documents'")
        os.makedirs(DOC_FOLDER, exist_ok=True)

Mounting Google Drive...
Mounted at /content/drive
âœ“ Google Drive mounted

ðŸ“ Select your documents folder below, then run Cell 2:
   (The picker is non-blocking - select BEFORE running the next cell)


FileChooser(path='/content/drive/MyDrive', filename='', title='Select your documents folder in Google Drive', …

In [4]:
# =============================================================================
# CELL 2: CONFIRM SELECTION AND LIST DOCUMENTS
# =============================================================================
# If you used a folder picker above, make sure you selected a folder
# BEFORE running this cell. The picker is non-blocking.
# =============================================================================

# Read selection from folder picker (if one was used)
if folder_chooser is not None and folder_chooser.selected_path:
    DOC_FOLDER = folder_chooser.selected_path
    print(f"âœ“ Using selected folder: {DOC_FOLDER}")
elif folder_chooser is not None:
    print("âš  No folder selected in picker!")
    print("  Please go back to Cell 1, select a folder, then run this cell again.")
else:
    # No picker used (upload or manual path)
    print(f"âœ“ Using folder: {DOC_FOLDER}")

# Confirm folder (listing skipped for speed)
doc_path = Path(DOC_FOLDER)
if doc_path.exists():
    print(f"âœ“ Folder set: {doc_path.absolute()}")
    print("  Run the next cells to load, chunk, and index documents.")
else:
    print(f"âš  Folder not found: {DOC_FOLDER}")
    print("  Please set DOC_FOLDER in the previous cell and run it again.")

âœ“ Using selected folder: /content/drive/MyDrive/Colab Notebooks/Corpora/ModelTService
âœ“ Folder set: /content/drive/MyDrive/Colab Notebooks/Corpora/ModelTService
  Run the next cells to load, chunk, and index documents.


---
## Stage 1: Document Loading

We need to extract text from our documents. For PDFs with embedded text,
PyMuPDF (fitz) reads the text layer directly - no OCR needed.

**Corpora:** Use PDFs from `Corpora/<name>/pdf_embedded/`. The `.txt` files in `txt/` are for checking retrieval vs OCR issues.

In [5]:
# Exercise 1 (and reuse): Official query lists. Reference: CR Jan 13, 20, 21, 23, 2026.
QUERIES_MODEL_T = [
    "How do I adjust the carburetor on a Model T?",
    "What is the correct spark plug gap for a Model T Ford?",
    "How do I fix a slipping transmission band?",
    "What oil should I use in a Model T engine?",
]
QUERIES_CR = [
    "What did Mr. Flood have to say about Mayor David Black in Congress on January 13, 2026?",
    "What mistake did Elise Stefanik make in Congress on January 23, 2026?",
    "What is the purpose of the Main Street Parity Act?",
    "Who in Congress has spoken for and against funding of pregnancy centers?",
]

In [6]:
import fitz  # PyMuPDF
from typing import List, Tuple

def load_text_file(filepath: str) -> str:
    """Load a plain text file."""
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
        return f.read()


def load_pdf_file(filepath: str) -> str:
    """
    Extract text from a PDF with embedded text.

    PyMuPDF reads the text layer directly.
    For scanned PDFs without embedded text, you'd need OCR.
    """
    doc = fitz.open(filepath)
    text_parts = []

    for page_num, page in enumerate(doc):
        text = page.get_text()
        if text.strip():
            # Add page marker for debugging/citation
            text_parts.append(f"\n[Page {page_num + 1}]\n{text}")

    doc.close()
    return "\n".join(text_parts)


def load_documents(doc_folder: str) -> List[Tuple[str, str]]:
    """Load all documents from a folder. Returns list of (filename, content)."""
    documents = []
    folder = Path(doc_folder)

    for filepath in folder.rglob("*"):
        try:
            if not filepath.is_file():
                continue
        except OSError:
            continue
        if filepath.suffix.lower() not in ('.pdf', '.txt', '.md', '.text'):
            continue
        try:
            if filepath.suffix.lower() == '.pdf':
                content = load_pdf_file(str(filepath))
            elif filepath.suffix.lower() in ['.txt', '.md', '.text']:
                content = load_text_file(str(filepath))
            else:
                continue

            if content.strip():
                documents.append((filepath.name, content))
                print(f"âœ“ Loaded: {filepath.name} ({len(content):,} chars)")
        except Exception as e:
            print(f"âœ— Error loading {filepath}: {e}")

    return documents

In [7]:
# Load your documents
documents = load_documents(DOC_FOLDER)
print(f"\nLoaded {len(documents)} documents")

if len(documents) == 0:
    print("\nâš  No documents loaded! Please add PDF or TXT files to the documents folder.")

âœ“ Loaded: Ford-Model-T-Man-1919.txt (95,574 chars)
âœ“ Loaded: ModelT-01-10.txt (18,676 chars)
âœ“ Loaded: ModelT-11-20.txt (19,009 chars)
âœ“ Loaded: ModelT-21-30.txt (17,050 chars)
âœ“ Loaded: ModelT-31-40.txt (12,194 chars)
âœ“ Loaded: ModelT-41-50.txt (14,264 chars)
âœ“ Loaded: ModelT-51-60.txt (14,168 chars)
âœ“ Loaded: ModelT-61-62.txt (201 chars)
âœ“ Loaded: Ford-Model-T-Man-1919-ocr.pdf (95,517 chars)
âœ“ Loaded: ModelT-01-10-ocr.pdf (18,658 chars)
âœ“ Loaded: ModelT-11-20-ocr.pdf (19,003 chars)
âœ“ Loaded: ModelT-21-30-ocr.pdf (17,025 chars)
âœ“ Loaded: ModelT-31-40-ocr.pdf (12,201 chars)
âœ“ Loaded: ModelT-41-50-ocr.pdf (14,270 chars)
âœ“ Loaded: ModelT-51-60-ocr.pdf (14,107 chars)
âœ“ Loaded: ModelT-61-62-ocr.pdf (204 chars)

Loaded 16 documents


In [8]:
# Inspect a document to verify loading worked
if documents:
    filename, content = documents[0]
    print(f"First document: {filename}")
    print(f"Total length: {len(content):,} characters")
    print(f"\nFirst 1000 characters:\n{'-'*40}")
    print(content[:1000])

First document: Ford-Model-T-Man-1919.txt
Total length: 95,574 characters

First 1000 characters:
----------------------------------------
For Owners and Operators

of Ford Cars and Trucks

Published by J
FORD MOTOR. COMPANY
Detroit, Michigan, U.S. A. .



Ss Foreword

' is a significant fact that nearly all Ford cara are driven by
laymen—by owners, who in the. great majority of cases have
FOr no practical experience with things mechanical.

- ' Ford sttvice statichs distributed throughout the civilized world —
N ' where Sdfust i A wit

But while it is not imperative, it is, however, altogether desir-
able that every Ford owner should thoroughly understand his
car, With euch knowledge at his command he ia always master ;

; ‘The mastery of a thorough lmowledge of Ford construction is
by no means a difficult or time-consuming task. The Ford is the
simplest car made. It is easy to understand, and is not difficult
te keep in proper adjustment and repair.
and that there may be an authorita

---
## Stage 2: Chunking

Documents need to be split into pieces small enough to be relevant but large enough to carry meaning.

**Why overlap?** If a key sentence sits right at a chunk boundary, splitting without overlap might cut it in half. Overlap ensures that information near boundaries appears intact in at least one chunk.

**Experiment:** Try different chunk sizes (256, 512, 1024) and see how it affects retrieval!

In [9]:
from dataclasses import dataclass

@dataclass
class Chunk:
    """A chunk of text with metadata for tracing back to source."""
    text: str
    source_file: str
    chunk_index: int
    start_char: int
    end_char: int


def chunk_text(
    text: str,
    source_file: str,
    chunk_size: int = 512,
    chunk_overlap: int = 128
) -> List[Chunk]:
    """
    Split text into overlapping chunks.

    We try to break at sentence or paragraph boundaries
    to avoid cutting mid-thought.
    """
    chunks = []
    start = 0
    chunk_index = 0

    while start < len(text):
        end = start + chunk_size

        # Try to break at a good boundary
        if end < len(text):
            # Look for paragraph break first
            para_break = text.rfind('\n\n', start + chunk_size // 2, end)
            if para_break != -1:
                end = para_break + 2
            else:
                # Look for sentence break
                sentence_break = text.rfind('. ', start + chunk_size // 2, end)
                if sentence_break != -1:
                    end = sentence_break + 2

        chunk_text_str = text[start:end].strip()

        if chunk_text_str:
            chunks.append(Chunk(
                text=chunk_text_str,
                source_file=source_file,
                chunk_index=chunk_index,
                start_char=start,
                end_char=end
            ))
            chunk_index += 1

        # Move forward, accounting for overlap
        start = end - chunk_overlap
        if chunks and start <= chunks[-1].start_char:
            start = end  # Safety: ensure progress

    return chunks

In [11]:
# ============================================
# EXPERIMENT: Try different chunk sizes!
# ============================================
CHUNK_SIZE = 512      # Try: 256, 512, 1024
CHUNK_OVERLAP = 128   # Try: 64, 128, 256
# For Ex 7/8 use rebuild_pipeline() â€” see cell after FAISS index.

# Chunk all documents
all_chunks = []
for filename, content in documents:
    doc_chunks = chunk_text(content, filename, CHUNK_SIZE, CHUNK_OVERLAP)
    all_chunks.extend(doc_chunks)
    print(f"{filename}: {len(doc_chunks)} chunks")

print(f"\nTotal: {len(all_chunks)} chunks")

Ford-Model-T-Man-1919.txt: 326 chunks
ModelT-01-10.txt: 64 chunks
ModelT-11-20.txt: 66 chunks
ModelT-21-30.txt: 56 chunks
ModelT-31-40.txt: 44 chunks
ModelT-41-50.txt: 51 chunks
ModelT-51-60.txt: 46 chunks
ModelT-61-62.txt: 1 chunks
Ford-Model-T-Man-1919-ocr.pdf: 316 chunks
ModelT-01-10-ocr.pdf: 63 chunks
ModelT-11-20-ocr.pdf: 61 chunks
ModelT-21-30-ocr.pdf: 56 chunks
ModelT-31-40-ocr.pdf: 44 chunks
ModelT-41-50-ocr.pdf: 47 chunks
ModelT-51-60-ocr.pdf: 44 chunks
ModelT-61-62-ocr.pdf: 1 chunks

Total: 1286 chunks


In [12]:
# Inspect some chunks
if all_chunks:
    print("Sample chunks:")
    indices_to_show = [0, len(all_chunks)//2, -1] if len(all_chunks) > 2 else range(len(all_chunks))
    for i in indices_to_show:
        chunk = all_chunks[i]
        print(f"\n{'='*60}")
        print(f"Chunk {chunk.chunk_index} from {chunk.source_file}")
        print(f"{'='*60}")
        print(chunk.text[:300] + "..." if len(chunk.text) > 300 else chunk.text)

Sample chunks:

Chunk 0 from Ford-Model-T-Man-1919.txt
For Owners and Operators

of Ford Cars and Trucks

Published by J
FORD MOTOR. COMPANY
Detroit, Michigan, U.S. A. .



Ss Foreword

' is a significant fact that nearly all Ford cara are driven by
laymen—by owners, who in the. great majority of cases have
FOr no practical experience with things mechan...

Chunk 36 from ModelT-51-60.txt
ers, it indicates that the cell is not in good ofiter
and the tyatiery should be taken to a skilled service man for attention.

ms stat water be an Answer No. 135.

Aa nething but pure wate othe ces and doit often enough to keep the plates
covered at all times. The solution (electrolyte) should be m...

Chunk 0 from ModelT-61-62-ocr.pdf
[Page 1]
What It Is For
How to Disconnect
OPERATION
Alustments—What to Do vere... cong
Model T Truck
‘Worm, How Removed rar bo eee ne ceeaee ba eeeeeces reer er rrr
The Ford Starting and Highting System


---
## Stage 3: Embedding

Embeddings map text to dense vectors where **semantic similarity = geometric proximity**.

A sentence about "cardiac arrest" and one about "heart attack" will have similar embeddings even though they share no words.

**Note:** sentence-transformers does NOT auto-detect Apple MPS - we must pass the device explicitly.

In [13]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load embedding model
# Options:
# - "sentence-transformers/all-MiniLM-L6-v2": Fast, small (80MB), good quality
# - "BAAI/bge-small-en-v1.5": Better for retrieval, similar size

EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

print(f"Loading embedding model: {EMBEDDING_MODEL}")
print(f"Device: {DEVICE}")

# Must explicitly pass device for MPS support!
embed_model = SentenceTransformer(EMBEDDING_MODEL, device=DEVICE)
EMBEDDING_DIM = embed_model.get_sentence_embedding_dimension()
print(f"Embedding dimension: {EMBEDDING_DIM}")

Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
Device: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding dimension: 384


In [14]:
# DEMO: See how embeddings capture semantic similarity
test_sentences = [
    "The engine needs regular oil changes.",
    "Motor oil should be replaced periodically.",
    "The Senate convened at noon.",
    "Congress began its session at midday."
]

test_embeddings = embed_model.encode(test_sentences)

# Compute cosine similarity matrix
from numpy.linalg import norm

def cosine_sim(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

print("Cosine similarity matrix:")
print("\n" + " " * 40 + "  [0]    [1]    [2]    [3]")
for i, s1 in enumerate(test_sentences):
    sims = [cosine_sim(test_embeddings[i], test_embeddings[j]) for j in range(4)]
    print(f"[{i}] {s1[:35]:35} {sims[0]:.3f}  {sims[1]:.3f}  {sims[2]:.3f}  {sims[3]:.3f}")

print("\nâ†’ Notice: [0]-[1] are similar (both about oil), [2]-[3] are similar (both about Congress)")

Cosine similarity matrix:

                                          [0]    [1]    [2]    [3]
[0] The engine needs regular oil change 1.000  0.728  -0.045  -0.032
[1] Motor oil should be replaced period 0.728  1.000  0.014  0.035
[2] The Senate convened at noon.        -0.045  0.014  1.000  0.684
[3] Congress began its session at midda -0.032  0.035  0.684  1.000

â†’ Notice: [0]-[1] are similar (both about oil), [2]-[3] are similar (both about Congress)


In [15]:
# Embed all chunks - this may take a few minutes for large corpora
if all_chunks:
    print(f"Embedding {len(all_chunks)} chunks on {DEVICE}...")
    chunk_texts = [c.text for c in all_chunks]
    chunk_embeddings = embed_model.encode(chunk_texts, show_progress_bar=True)
    chunk_embeddings = chunk_embeddings.astype('float32')  # FAISS wants float32
    print(f"Embeddings shape: {chunk_embeddings.shape}")
else:
    print("No chunks to embed - please load documents first.")

Embedding 1286 chunks on cpu...


Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Embeddings shape: (1286, 384)


---
## Stage 4: Vector Index (FAISS)

FAISS efficiently finds nearest neighbors in high-dimensional spaces.

We use a simple **flat index** (brute-force search) which is transparent and works well for up to ~100k vectors. For larger corpora, you'd use approximate methods like IVF or HNSW.

**Note:** FAISS GPU support is CUDA-only. On MPS/CPU, we use faiss-cpu (still very fast for <100k vectors).

In [16]:
import faiss

# Create FAISS index
# IndexFlatIP = Inner Product (for cosine similarity on normalized vectors)
index = faiss.IndexFlatIP(EMBEDDING_DIM)

if all_chunks:
    # Normalize vectors so inner product = cosine similarity
    faiss.normalize_L2(chunk_embeddings)

    # Add vectors to index
    index.add(chunk_embeddings)
    print(f"Index built with {index.ntotal} vectors")
else:
    print("No embeddings to index - please load and embed documents first.")

Index built with 1286 vectors


---
## Stage 5: Retrieval

Now we can search! Given a query, we:
1. Embed the query with the same model
2. Find the top-k most similar chunks
3. Return those chunks as context

In [17]:
# Helper for Exercises 7 & 8: rebuild chunks + index with different chunk_size / chunk_overlap.
def rebuild_pipeline(chunk_size: int = 512, chunk_overlap: int = 128):
    """Re-chunk documents, re-embed, and rebuild FAISS index. Updates global all_chunks and index."""
    global all_chunks, index
    all_chunks = []
    for filename, content in documents:
        all_chunks.extend(chunk_text(content, filename, chunk_size=chunk_size, chunk_overlap=chunk_overlap))
    chunk_embeddings = embed_model.encode([c.text for c in all_chunks], show_progress_bar=True).astype("float32")
    faiss.normalize_L2(chunk_embeddings)
    index = faiss.IndexFlatIP(EMBEDDING_DIM)
    index.add(chunk_embeddings)
    print(f"Rebuilt: {len(all_chunks)} chunks, chunk_size={chunk_size}, chunk_overlap={chunk_overlap}")

In [18]:
def retrieve(query: str, top_k: int = 5):
    """
    Retrieve the top-k most relevant chunks for a query.

    Returns: List of (chunk, similarity_score) tuples
    """
    # Embed the query
    query_embedding = embed_model.encode([query]).astype('float32')
    faiss.normalize_L2(query_embedding)

    # Search
    scores, indices = index.search(query_embedding, top_k)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx != -1:
            results.append((all_chunks[idx], float(score)))

    return results

In [19]:
# Test retrieval
# ============================================
# TRY DIFFERENT QUERIES FOR YOUR CORPUS!
# ============================================
test_query = "What is the procedure for engine maintenance?"  # â† Modify this!

if index.ntotal > 0:
    results = retrieve(test_query, top_k=5)

    print(f"Query: {test_query}\n")
    print("Top 5 retrieved chunks:")
    for i, (chunk, score) in enumerate(results, 1):
        print(f"\n[{i}] Score: {score:.4f} | Source: {chunk.source_file}")
        print(f"    {chunk.text[:200]}...")
else:
    print("Index is empty - please load, chunk, and embed documents first.")

Query: What is the procedure for engine maintenance?

Top 5 retrieved chunks:

[1] Score: 0.4715 | Source: ModelT-01-10.txt
    ake a practice of taking care of every repair '
adjustment as soon as its necessity is discovered. This attention requires but

little time and may. avoid delay or possible accident on the road. We ai...

[2] Score: 0.4715 | Source: Ford-Model-T-Man-1919.txt
    ake a practice of taking care of every repair '
adjustment as soon as its necessity is discovered. This attention requires but

little time and may. avoid delay or possible accident on the road. We ai...

[3] Score: 0.4531 | Source: Ford-Model-T-Man-1919-ocr.pdf
    o far.
6. Gas mixture too rich.
7. Water circulation retarded ‘by
sediment in radiator. .
8 Dirty spark plugs.
ENGINE KNOCKS
1. Carbon deposit on piston heads.
2. Loose connecting rod bearing.
3. Loos...

[4] Score: 0.4486 | Source: Ford-Model-T-Man-1919.txt
    far.
6. Gas mixture too rich.

7. Water circulation retarded ‘by
sediment in ra

---
## Stage 6: Generation (LLM)

Now we load a local LLM to generate answers from the retrieved context.

**Recommended models:**
- `Qwen/Qwen2.5-1.5B-Instruct` - Best instruction following at this size
- `Qwen/Qwen2.5-3B-Instruct` - Even better if you have 8GB+ VRAM
- `meta-llama/Llama-3.2-1B-Instruct` - Alternative, slightly weaker

**Device handling:**
- CUDA: Uses `device_map="auto"` and float16
- MPS: Loads to CPU first, then moves to MPS with float32
- CPU: Uses float32 (slower but works)

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# ============================================
# CHOOSE YOUR MODEL
# ============================================
LLM_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"  # Or try "Qwen/Qwen2.5-3B-Instruct"

print(f"Loading LLM: {LLM_MODEL}")
print(f"Device: {DEVICE}, Dtype: {DTYPE}")
print("This may take a few minutes on first run...\n")

tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)

# Load with appropriate settings for each device type
if DEVICE == 'cuda':
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL,
        device_map="auto",
        torch_dtype=DTYPE,
        trust_remote_code=True
    )
    print("Model loaded on CUDA")

elif DEVICE == 'mps':
    # For MPS, load to CPU first, then move to MPS
    # (device_map="auto" doesn't work well with MPS)
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL,
        torch_dtype=DTYPE,
        trust_remote_code=True
    )
    model = model.to(DEVICE)
    print("Model loaded on MPS (Apple Silicon)")

else:
    # CPU
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL,
        torch_dtype=DTYPE,
        trust_remote_code=True
    )
    print("Model loaded on CPU (this will be slow)")

Loading LLM: Qwen/Qwen2.5-1.5B-Instruct
Device: cpu, Dtype: torch.float32
This may take a few minutes on first run...



config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Model loaded on CPU (this will be slow)


In [21]:
def generate_response(prompt: str, max_new_tokens: int = 512, temperature: float = 0.3) -> str:
    """
    Generate a response from the LLM.

    Lower temperature = more focused/deterministic
    Higher temperature = more creative/random
    """
    inputs = tokenizer(prompt, return_tensors="pt")

    # Move inputs to the correct device
    if DEVICE == 'cuda':
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
    else:
        inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True if temperature > 0 else False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode only the new tokens
    response = tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    )

    return response.strip()

---
## Stage 7: The Complete RAG Pipeline

Now we put it all together. The **prompt template** is critical - it must instruct the model to use the retrieved context.

In [22]:
# The RAG prompt template
PROMPT_TEMPLATE = """You are a helpful assistant that answers questions based on the provided context.

CONTEXT:
{context}

QUESTION: {question}

INSTRUCTIONS:
- Answer the question based ONLY on the information in the context above
- If the context doesn't contain enough information to answer, say so
- Quote relevant parts of the context to support your answer
- Be concise and direct

ANSWER:"""


def direct_query(question: str, max_new_tokens: int = 512) -> str:
    """Ask the LLM directly with no retrieved context (for RAG vs no-RAG comparison)."""
    prompt = f"""Answer this question:
{question}

Answer:"""
    return generate_response(prompt, max_new_tokens=max_new_tokens)

def rag_query(question: str, top_k: int = 5, show_context: bool = False, prompt_template: str = None) -> str:
    """The complete RAG pipeline. prompt_template: custom template for Exercise 10."""
    # Step 1: Retrieve
    results = retrieve(question, top_k)

    # Format context
    context_parts = []
    for chunk, score in results:
        context_parts.append(f"[Source: {chunk.source_file}, Relevance: {score:.3f}]\n{chunk.text}")
    context = "\n\n---\n\n".join(context_parts)

    if show_context:
        print("=" * 60)
        print("RETRIEVED CONTEXT:")
        print("=" * 60)
        print(context)
        print("=" * 60 + "\n")

    # Step 2: Build prompt (use custom template if provided)
    template = prompt_template if prompt_template is not None else PROMPT_TEMPLATE
    prompt = template.format(context=context, question=question)

    # Step 3: Generate
    answer = generate_response(prompt)

    return answer

In [23]:
# ============================================
# TEST YOUR RAG PIPELINE!
# ============================================

question = "What maintenance is required for the engine?"  # â† Modify for your corpus!

if index.ntotal > 0:
    print(f"Question: {question}\n")
    print("Generating answer...\n")

    answer = rag_query(question, top_k=5, show_context=True)

    print("ANSWER:")
    print(answer)
else:
    print("Pipeline not ready - please complete all previous stages first.")

Question: What maintenance is required for the engine?

Generating answer...

RETRIEVED CONTEXT:
[Source: ModelT-01-10-ocr.pdf, Relevance: 0.494]
.
entire Ford orgenization is interested in keeping every individual Ford. car
constant operation, at the lowest possible cost. We have known of much dé
done to miany cars by unskilled repair men.
What attention. Z
does the Car need? Answer No. vA
mer that a new machine requires more careful attention during the fiz
few é itis being driven than efter the parts have become thoroughly “worked!
‘he car which is driven slowly and carefully when new usually gives thei
aure that is has plenty of oil and water.

---

[Source: Ford-Model-T-Man-1919-ocr.pdf, Relevance: 0.494]
.
entire Ford orgenization is interested in keeping every individual Ford. car
constant operation, at the lowest possible cost. We have known of much dé
done to miany cars by unskilled repair men.
What attention. Z
does the Car need? Answer No. vA
mer that a new machine requires 

---
## Experiments: Understanding RAG Behavior

Now that you have a working pipeline, try these experiments to understand how each component affects the results.

In [24]:
# EXPERIMENT 1: Compare WITH vs WITHOUT RAG
# ==========================================

question = "What are the specifications for the landing gear?"  # â† Use a corpus-specific question!

if index.ntotal > 0:
    # WITHOUT RAG - just ask the model directly
    direct_prompt = f"""Answer this question:
{question}

Answer:"""

    print("WITHOUT RAG (model's own knowledge):")
    print("-" * 40)
    direct_answer = generate_response(direct_prompt)
    print(direct_answer)

    print("\n" + "=" * 60 + "\n")

    # WITH RAG
    print("WITH RAG (using retrieved context):")
    print("-" * 40)
    rag_answer = rag_query(question, top_k=5)
    print(rag_answer)
else:
    print("Please complete the pipeline setup first.")

WITHOUT RAG (model's own knowledge):
----------------------------------------
The landing gear is a set of wheels that allow an aircraft to take off and land on a runway. It consists of two main parts - the nose wheel and the main wheels.
The landing gear is typically made from aluminum alloy or steel, and it can be either fixed or retractable. Fixed landing gears do not fold up when the plane is parked, while retractable landing gears fold down into the fuselage when the plane is parked.
Landing gear specifications include things like maximum lift-off weight, maximum landing weight, and maximum landing speed. These specifications help ensure that the aircraft can safely take off and land under normal conditions, as well as in emergency situations such as engine failure or loss of control. Answer this question:

What are the specifications for the landing gear? To answer this question, I'll follow these steps:

1. Identify the key components of the landing gear
2. Determine what inform

In [25]:
# EXPERIMENT 2: Effect of top_k
# ==========================================

question = "What safety procedures are required?"  # â† Use a corpus-specific question!

if index.ntotal > 0:
    for k in [1, 3, 5, 10]:
        print(f"\n{'='*60}")
        print(f"TOP_K = {k}")
        print(f"{'='*60}")
        answer = rag_query(question, top_k=k)
        print(answer[:500] + "..." if len(answer) > 500 else answer)
else:
    print("Please complete the pipeline setup first.")


TOP_K = 1
According to the text, some safety procedures include:

- Driving slowly and carefully instead of quickly when changing a flat tire,
- Not locking the wheels with the brakes as it strains them,
- Avoiding running over streetcar tracks, ruts, or bumping the side of the tire against curb stones. 

The text also mentions painting the wheel rims each season to prevent rust. However, there is no mention of other specific safety measures beyond these points. To provide more detailed safety procedures ...

TOP_K = 3
Before starting the car, you must fill the radiator with clean, fresh water. If it's not possible to obtain perfect clean water, you can strain it through a cloth or similar material to avoid any debris from entering the radiator and potentially blocking its tubes. This ensures the cooling system remains clear and efficient for optimal performance. To summarize:

1. Fill the radiator with clean, fresh water.
2. Strain the water if it's not perfectly clean to prevent blo

In [26]:
# EXPERIMENT 3: Question the corpus CAN'T answer
# ==========================================
# Does the model admit it doesn't know, or hallucinate?

unanswerable_question = "What is the CEO's favorite color?"

if index.ntotal > 0:
    print(f"Question: {unanswerable_question}\n")
    answer = rag_query(unanswerable_question, top_k=5, show_context=True)
    print(f"\nAnswer: {answer}")
else:
    print("Please complete the pipeline setup first.")

Question: What is the CEO's favorite color?

RETRIEVED CONTEXT:
[Source: ModelT-01-10-ocr.pdf, Relevance: 0.185]
[Page 1]
For Owners and Operators
of Ford Cars and Trucks
Published by J
FORD MOTOR. COMPANY
Detroit, Michigan, U.S. A. .


[Page 2]
Ss Foreword
' is a significant fact that nearly all Ford cara are driven by
laymen—by owners, who in the. great majority of cases have
FOr no practical experience with things mechanical.
- ' Ford sttvice statichs distributed throughout the civilized world —
N ' where Sdfust i A wit
But while it is not imperative, it is, however, altogether desir-
able that every Ford owner sh

---

[Source: Ford-Model-T-Man-1919-ocr.pdf, Relevance: 0.185]
[Page 1]
For Owners and Operators
of Ford Cars and Trucks
Published by J
FORD MOTOR. COMPANY
Detroit, Michigan, U.S. A. .


[Page 2]
Ss Foreword
' is a significant fact that nearly all Ford cara are driven by
laymen—by owners, who in the. great majority of cases have
FOr no practical experience with things mec

---
## Save/Load Your Index

For large corpora, you don't want to re-embed every time. Here's how to persist the index.

In [27]:
import pickle

def save_index(filepath: str):
    """Save FAISS index and chunks to disk."""
    faiss.write_index(index, f"{filepath}.faiss")
    with open(f"{filepath}.chunks", 'wb') as f:
        pickle.dump(all_chunks, f)
    print(f"âœ“ Saved index to {filepath}.faiss")
    print(f"âœ“ Saved chunks to {filepath}.chunks")

def load_saved_index(filepath: str):
    """Load FAISS index and chunks from disk."""
    global index, all_chunks
    index = faiss.read_index(f"{filepath}.faiss")
    with open(f"{filepath}.chunks", 'rb') as f:
        all_chunks = pickle.load(f)
    print(f"âœ“ Loaded index with {index.ntotal} vectors")

# Save your index
if index.ntotal > 0:
    save_index("my_rag_index")
else:
    print("No index to save.")

# Later, to load:
# load_saved_index("my_rag_index")

âœ“ Saved index to my_rag_index.faiss
âœ“ Saved chunks to my_rag_index.chunks


---
## Next Steps

You've built a complete RAG pipeline from scratch! In the next class, we'll:

1. **Improve retrieval** with query rewriting and hybrid search
2. **Rebuild with LangChain** to see how frameworks abstract these steps
3. **Evaluate systematically** with test questions and metrics

### Exercises to try:
- Vary chunk size (256, 512, 1024) and measure retrieval quality
- Try a different embedding model (`BAAI/bge-small-en-v1.5`)
- Try a larger LLM (`Qwen/Qwen2.5-3B-Instruct`) and compare answer quality
- Ask questions that require combining information from multiple chunks

---
## Appendix: Device Information

Run this cell to see detailed information about your compute environment.

In [28]:
def print_device_info():
    """Print detailed information about available compute devices."""
    print("=" * 60)
    print("DEVICE INFORMATION")
    print("=" * 60)

    print(f"\nEnvironment: {ENVIRONMENT}")
    print(f"PyTorch version: {torch.__version__}")

    # CUDA
    print(f"\nCUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"  Device: {torch.cuda.get_device_name(0)}")
        print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    # MPS
    print(f"\nMPS available: {torch.backends.mps.is_available()}")
    print(f"MPS built: {torch.backends.mps.is_built()}")

    # Current selection
    print(f"\nâ†’ Selected device: {DEVICE}")
    print(f"â†’ Selected dtype: {DTYPE}")
    print("=" * 60)

print_device_info()

DEVICE INFORMATION

Environment: colab
PyTorch version: 2.9.0+cpu

CUDA available: False

MPS available: False
MPS built: False

â†’ Selected device: cpu
â†’ Selected dtype: torch.float32
